# Detect and blur faces in video using Amazon Rekognition Video

***
This notebook provides a walkthrough of [face detection API](https://docs.aws.amazon.com/rekognition/latest/APIReference/API_StartFaceDetection.html) in Amazon Rekognition Video to identify faces in a stored video.
***

## Initialization

In [ ]:
!pip install opencv-python boto3 botocore ffmpeg-python moviepy --upgrade -q

In [ ]:
import boto3
import botocore

In [ ]:
client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account:{account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region:{region}")

In [ ]:
# S3 bucket that will be the location from where we will read images/videos

bucket = 'dml102-20221003-sagemaker-bucket-{region}-{account}'.format(region = region, account = account)

In [ ]:
# copy video from local EBS storage to S3 bucket

!aws s3 cp videos/people-walking.mp4 s3://$bucket/rekognition-video-demo/people-walking.mp4

## Detect faces in video
***

Explore the code in blur_faces/

In [ ]:
from blur_faces.rekognition import get_timestamps_and_faces
from blur_faces.rekognition import boto3_client, start_face_detection
from blur_faces.rekognition import wait_for_completion
from blur_faces.video_processor import apply_faces_to_video, integrate_audio

Amazon Rekognition reads the video from an S3 bucket.

In [ ]:
video = 'rekognition-video-demo/people-walking.mp4'
local_path_to_video = 'videos/people-walking.mp4'
local_output = 'videos/output.mp4'

In [ ]:
reko = boto3_client()
job_id = start_face_detection(bucket, video, 1, reko)
response = wait_for_completion(job_id, reko_client=reko)
timestamps=get_timestamps_and_faces(response, job_id, reko)
apply_faces_to_video(timestamps, local_path_to_video, local_output, response["VideoMetadata"])
# integrate_audio('videos/video-test.mp4', 'videos/output.mp4')